In [2]:
import numpy as np
import pandas as pd


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack
from scipy.special import logit, expit
from tqdm import tqdm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('../input/train.csv').fillna(' ')
test = pd.read_csv('../input/test.csv').fillna(' ')
# train = train[0:10000]
# test = test[0:10000]
print(train)
train_text = train['comment_text']
test_text = test['comment_text']

# train_text = train_text[0:100]
# test_text = test_text[0:100]
def correlation(a , b ,unwanted='*'):
	a = list(a.lower())
	b = list(b.lower())
	
	count = 0 
	flag =0
	for i , j in zip(a,b) :
		if (not flag) and j not in unwanted:
			flag =1
		if i == j or j in unwanted :
			count = count + 1
			continue

		return 0
	return count >= len(a)/2 and flag
print(train.head())

In [3]:

def remove_unwanted_char(arr):
    badWords = ["fuck" ,"cock" ,"suck" ,"piss" ,"bullshit" ,"ass" ,"asshole" ,"dick","shit" ,"motherfuck"]
    for k in range(len(arr)) :
        tmp = arr[k].split()
        flag = 0
        for j in badWords:
            for i in range(len(tmp)):
                if correlation(j ,tmp[i]) :
                    tmp[i]= j
                    flag =1
                    
        if flag :
            arr[k] = " ".join(tmp)
    return arr
#train_text = remove_unwanted_char(train_text)
#test_text = remove_unwanted_char(test_text)

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Dense, Dropout, Activation ,LSTM

def new_model(  drop):
    model = Sequential()

    # Add an input layer 
    model.add(Dense(300, activation='relu', input_shape=(3000,)))
    model.add(Dropout(drop))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(drop))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [5]:


all_text = pd.concat([train_text])
 




losses = []

word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}.',
    ngram_range=(1, 1),
    max_features=3000,
    max_df =.4 ,
    min_df    = .000001
)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
# test_word_features = word_vectorizer.transform(test_text)
# train_features = hstack([train_word_features])
# test_features = hstack([test_char_features,test_word_features])


In [6]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Dense, Dropout, Activation ,LSTM


train_label = train["toxic"]
split_point = int((2/3)*len(train_label))

print(train_word_features.shape)
x_train = (train_word_features.toarray())[:split_point]
x_test = (train_word_features.toarray())[split_point:]

y_train = train_label[:split_point]
y_test = train_label[split_point:]


In [17]:
model = new_model(.5)
model.fit(x_train,y_train,epochs=100,batch_size=10)
testPredict = model.predict_classes(x_test)


In [19]:
count =0
y_test = np.asarray(y_test)

for i in range(len(testPredict)):
    if(testPredict[i]>=0.5 and y_test[i]==1 or testPredict[i]<=0.5 and y_test[i]==0 ):
        count=count+1
print(count/len(y_test))

In [ ]:
# from keras.models import Sequential
# from keras.layers import Dense, Activation
# from keras.layers import Dense, Dropout, Activation ,LSTM

# def new_model(  drop):
 
#     model = Sequential()  
#     model.add(LSTM(100, input_shape=(400, 100),return_sequences=True))
#     model.add(Dense(100))
#     model.compile(loss='mean_absolute_error', optimizer='adam',metrics=['accuracy'])

#     return model


In [ ]:
# train_features = [train_features.toarray().T]

# train_labels = train[""]
predictions = {'id': test['id']}
for class_name in (class_names):
#     if class_name == "threat" :
        
#     train_target = [[list(train[class_name])]]
    train_target = train[class_name]
    model = new_model(.5)
   
    #classifier = LogisticRegression(solver='sag',multi_class="ovr",tol=1e-10)
#     classifier =LinearDiscriminantAnalysis(solver=’svd’, shrinkage=None, priors=None, n_components=None, store_covariance=False, tol=0.0001)
#     cv_loss = np.mean(cross_val_score(classifier,train_features, train_target, cv=3, scoring='roc_auc'))
#     losses.append(cv_loss)
#     print('CV score for class {} is {}'.format(class_name, cv_loss))
    print(np.shape(train_features) ,np.shape(train_target))
    model.fit(train_features, train_target,epochs=7, batch_size=50, verbose=1)
    predictions[class_name] = model.predict( test_features )
#     classifier.fit(train_features, train_target)
#     predictions[class_name] = classifier.predict_proba(test_features)[:, 1]
    
print('Total CV score is {}'.format(np.mean(losses)))
submission = pd.DataFrame.from_dict(predictions)
submission.to_csv('submission.csv', index=False)

 

In [ ]:
print(np.shape(train_features))

In [ ]:
# from scipy import sparse
# sparse.save_npz('train_featureMatrix.npz', train_features)
# sparse.save_npz('test_featureMatrix.npz', test_features)

# #Load
# #data = sparse.load_npz("data_sparse.npz")
# np.savetxt('train_featureMatrix.csv',train_features,delimiter=',' , fmt='%.10f' )
# np.savetxt('test_featureMatrix.csv',test_features,delimiter=',' , fmt='%.10f' )